In [1]:
import eranest

In [ ]:
# eranest.era5ify_geojson(request_id, variables, start_date, end_date, json_file, dataset_type, pressure_levels, frequency, resolution)
# request_id : str, unique identifier for the request
# variables : list, list of variables to download
# start_date : string, start date of the data ('YYYY-M-D' or 'YYYY-MM-DD')
# end_date : string, end date of the data ('YYYY-M-D' or 'YYYY-MM-DD')
# json_file : str, path to the geojson file containing the area of interest
# dataset_type : str, type of dataset (single or pressure, for single level or pressure level datasets), optional (single by default)
# pressure_levels : list, list of strings of pressure levels to download (e.g., ["1000", "925", "850"]), optional (empty by default)
# frequency : str, frequency of the data (hourly, daily, weekly, monthly, yearly), optional (hourly by default)
# resolution : float, resolution of the data in degrees (0.1, 0.25, etc.), optional (0.25 by default)

In [2]:
df = eranest.era5ify_geojson(
    request_id="test",
    variables=[
        "2m_temperature",
        "total_precipitation",
        "surface_pressure",
        "2m_dewpoint_temperature",
    ],
    start_date="2023-1-1",
    end_date="2023-1-31", 
    json_file="../data/india.json",
    dataset_type="single",
    frequency="daily",
    resolution="0.25",
)

✓ CDS API configuration is already set up and valid.

STARTING ERA5 SINGLE LEVEL PROCESSING
Request ID: test
Variables: ['2m_temperature', 'total_precipitation', 'surface_pressure', '2m_dewpoint_temperature']
Date Range: 2023-01-01 to 2023-01-31
Frequency: daily
Resolution: 0.25°
GeoJSON File: C:\Users\ATHARV~1\AppData\Local\Temp\test_temp_geojson.json
✓ All inputs validated successfully

--- Bounding Box ---
✓ Bounding Box calculated:
  North: 35.4940°
  South: 7.9655°
  East:  97.4026°
  West:  68.1766°
  Area:  29.2259° × 27.5285°


--- GeoJSON Mini Map ---

MINI MAP (68.18°W to 97.40°E, 7.97°S to 35.49°N):
┌─────────────────────────────────────────┐
│·········································│
│········■■■■■■■··························│
│··········■■■■■··························│
│·········■■■■■■■·························│
│·······■■■■■■■■■■························│
│····■■■■■■■■■■■■■■■■········■····■■■■■■■·│
│···■■■■■■■■■■■■■■■■■■■■■■■■■·■■■■■■■■····│
│····■■■■■■■■■■■■■■■■■■■■■■■■·

2025-07-11 10:08:23,074 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
INFO:datapi.legacy_api_client:[2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-07-11 10:08:24,060 INFO Request ID is d0948d93-3257-4517-888c-03a554cd540d
INFO:datapi.legacy_api_client:Request ID is d0948d93-3257-4517-888c-03a554cd540d
2025-07-11 10:08:24,387 INFO status has been updated to accepted
INFO:datapi.legacy_api_client:status has been updated to accepted
2025-07-11 10:08:33,623 INFO status has been updated to running
INFO:datapi.legacy_api_client:status has been updated to running
2025-07-11 10:08:46,640 INFO status has been updated to successful
INFO:datapi.legacy_api_client:status has been updated to successful
                                                                                          

  ✓ Download completed: C:\Users\ATHARV~1\AppData\Local\Temp\test_chunk1.zip
Extracting zip file: C:\Users\ATHARV~1\AppData\Local\Temp\test_chunk1.zip
Extracted NetCDF files:
  - C:\Users\ATHARV~1\AppData\Local\Temp\test_chunk1\data_stream-oper_stepType-accum.nc
  - C:\Users\ATHARV~1\AppData\Local\Temp\test_chunk1\data_stream-oper_stepType-instant.nc

Processing downloaded data:
- Found 2 file(s)
  Processing file 1/2: data_stream-oper_stepType-accum.nc


  ✓ Loaded: Dimensions: Frozen({'valid_time': 336, 'latitude': 111, 'longitude': 117})
  Processing file 2/2: data_stream-oper_stepType-instant.nc
  ✓ Loaded: Dimensions: Frozen({'valid_time': 336, 'latitude': 111, 'longitude': 117})
Starting optimized filtering process...
→ Extracting unique lat/lon coordinates from dataset...
✓ Found 12987 unique lat/lon combinations
→ Filtering unique coordinates against polygon...
✓ Coordinate filtering completed in 0.50 seconds
  - Points inside: 4446
  - Points outside: 8541
  - Percentage inside: 34.23%
→ Filtering original dataset using inside coordinates...
  Converting dataset to DataFrame...
  ✓ Converted to DataFrame with 4363632 rows
  ✓ Created lookup set with 4446 coordinate pairs
  Filtering DataFrame rows...
  ✓ Filtered from 4363632 to 1493856 rows
✓ Dataset filtering completed in 3.03 seconds

--- Final Filtering Results ---
Total processing time: 3.54 seconds
Final DataFrame shape: (1493856, 9)
Rows in final dataset: 1493856
✓ Chunk

2025-07-11 10:09:25,947 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
INFO:datapi.legacy_api_client:[2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-07-11 10:09:27,074 INFO Request ID is 81505e3b-73e8-4dce-b5ab-3d99928c196b
INFO:datapi.legacy_api_client:Request ID is 81505e3b-73e8-4dce-b5ab-3d99928c196b
2025-07-11 10:09:27,259 INFO status has been updated to accepted
INFO:datapi.legacy_api_client:status has been updated to accepted
2025-07-11 10:09:36,495 INFO status has been updated to running
INFO:datapi.legacy_api_client:status has been updated to running
2025-07-11 10:09:41,718 INFO status has been updated to successful
INFO:datapi.legacy_api_client:status has been updated to successful
                                                                                          

  ✓ Download completed: C:\Users\ATHARV~1\AppData\Local\Temp\test_chunk2.zip
Extracting zip file: C:\Users\ATHARV~1\AppData\Local\Temp\test_chunk2.zip
Extracted NetCDF files:
  - C:\Users\ATHARV~1\AppData\Local\Temp\test_chunk2\data_stream-oper_stepType-accum.nc
  - C:\Users\ATHARV~1\AppData\Local\Temp\test_chunk2\data_stream-oper_stepType-instant.nc

Processing downloaded data:
- Found 2 file(s)
  Processing file 1/2: data_stream-oper_stepType-accum.nc
  ✓ Loaded: Dimensions: Frozen({'valid_time': 336, 'latitude': 111, 'longitude': 117})
  Processing file 2/2: data_stream-oper_stepType-instant.nc
  ✓ Loaded: Dimensions: Frozen({'valid_time': 336, 'latitude': 111, 'longitude': 117})


Starting optimized filtering process...
→ Extracting unique lat/lon coordinates from dataset...
✓ Found 12987 unique lat/lon combinations
→ Filtering unique coordinates against polygon...
✓ Coordinate filtering completed in 0.53 seconds
  - Points inside: 4446
  - Points outside: 8541
  - Percentage inside: 34.23%
→ Filtering original dataset using inside coordinates...
  Converting dataset to DataFrame...
  ✓ Converted to DataFrame with 4363632 rows
  ✓ Created lookup set with 4446 coordinate pairs
  Filtering DataFrame rows...
  ✓ Filtered from 4363632 to 1493856 rows
✓ Dataset filtering completed in 2.98 seconds

--- Final Filtering Results ---
Total processing time: 3.52 seconds
Final DataFrame shape: (1493856, 9)
Rows in final dataset: 1493856
✓ Chunk completed in 43.1 seconds
PROCESSING CHUNK 3/3
Date Range: 2023-01-29 to 2023-01-31
Variables:  2m_temperature, total_precipitation, surface_pressure, 2m_dewpoint_temperature
  → Downloading ERA5 data (attempt 1/6)...


2025-07-11 10:10:20,118 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
INFO:datapi.legacy_api_client:[2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-07-11 10:10:21,041 INFO Request ID is c3e92e07-a435-4d74-a3c8-0d614ac62db9
INFO:datapi.legacy_api_client:Request ID is c3e92e07-a435-4d74-a3c8-0d614ac62db9
2025-07-11 10:10:21,244 INFO status has been updated to accepted
INFO:datapi.legacy_api_client:status has been updated to accepted
2025-07-11 10:10:30,370 INFO status has been updated to running
INFO:datapi.legacy_api_client:status has been updated to running
2025-07-11 10:10:35,610 INFO status has been updated to successful
INFO:datapi.legacy_api_client:status has been updated to successful
                                                                                         

  ✓ Download completed: C:\Users\ATHARV~1\AppData\Local\Temp\test_chunk3.zip
Extracting zip file: C:\Users\ATHARV~1\AppData\Local\Temp\test_chunk3.zip
Extracted NetCDF files:
  - C:\Users\ATHARV~1\AppData\Local\Temp\test_chunk3\data_stream-oper_stepType-accum.nc
  - C:\Users\ATHARV~1\AppData\Local\Temp\test_chunk3\data_stream-oper_stepType-instant.nc

Processing downloaded data:
- Found 2 file(s)
  Processing file 1/2: data_stream-oper_stepType-accum.nc
  ✓ Loaded: Dimensions: Frozen({'valid_time': 72, 'latitude': 111, 'longitude': 117})
  Processing file 2/2: data_stream-oper_stepType-instant.nc
  ✓ Loaded: Dimensions: Frozen({'valid_time': 72, 'latitude': 111, 'longitude': 117})


Starting optimized filtering process...
→ Extracting unique lat/lon coordinates from dataset...
✓ Found 12987 unique lat/lon combinations
→ Filtering unique coordinates against polygon...
✓ Coordinate filtering completed in 0.49 seconds
  - Points inside: 4446
  - Points outside: 8541
  - Percentage inside: 34.23%
→ Filtering original dataset using inside coordinates...
  Converting dataset to DataFrame...
  ✓ Converted to DataFrame with 935064 rows
  ✓ Created lookup set with 4446 coordinate pairs
  Filtering DataFrame rows...
  ✓ Filtered from 935064 to 320112 rows
✓ Dataset filtering completed in 0.60 seconds

--- Final Filtering Results ---
Total processing time: 1.10 seconds
Final DataFrame shape: (320112, 9)
Rows in final dataset: 320112
✓ Chunk completed in 31.3 seconds
AGGREGATING DATA (DAILY)
Aggregating data to daily frequency...
Sum columns: ['tp']
Max columns: []
Min columns: []
Rate columns: []
Average columns: ['t2m', 'sp', 'd2m']
Aggregation completed in:   0.64 seconds


In [3]:
df = eranest.era5ify_geojson(
    request_id="test2",
    variables=[
        "2m_temperature", 
        "total_precipitation"
    ],
    start_date="2024-1-1",
    end_date="2024-12-31",
    json_file="../data/latvia.geojson",
    dataset_type="single",
    frequency="monthly",
    resolution="0.1",
)

✓ CDS API configuration is already set up and valid.

STARTING ERA5 SINGLE LEVEL PROCESSING
Request ID: test2
Variables: ['2m_temperature', 'total_precipitation']
Date Range: 2024-01-01 to 2024-12-31
Frequency: monthly
Resolution: 0.1°
GeoJSON File: C:\Users\ATHARV~1\AppData\Local\Temp\test2_temp_geojson.json
✓ All inputs validated successfully

--- Bounding Box ---
✓ Bounding Box calculated:
  North: 58.0856°
  South: 55.6776°
  East:  28.2431°
  West:  20.9537°
  Area:  7.2894° × 2.4080°




--- GeoJSON Mini Map ---

MINI MAP (20.95°W to 28.24°E, 55.68°S to 58.09°N):
┌────────────────────────────────────────────────────────────┐
│····························································│
│····························■■■■■■■■■■■■■■··················│
│······■■■■■■■■■■············■■■■■■■■■■■■■■■■■■■■■■■■■■······│
│····■■■■■■■■■■■■■■■········■■■■■■■■■■■■■■■■■■■■■■■■■■■■■····│
│··■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■·····│
│·■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■·│
│·■■■■·····················■·■····■■■■■■■■■■■■■■■■■■■■■■■■■■·│
│·········································■■■■■■■■■■■■■■■····│
│····························································│
└────────────────────────────────────────────────────────────┘
 ■ = Inside the shape
 · = Outside the shape

--- Processing Strategy ---
Using monthly dataset: True
Total months to process: 12
Max months per chunk: 100
Needs chunking: False
  → Downloading ERA5 data (attempt 1/6)...


2025-07-11 11:05:04,951 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
INFO:datapi.legacy_api_client:[2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-07-11 11:05:05,568 INFO Request ID is 6f8fd806-91e0-4a19-abcd-de91e24fde92
INFO:datapi.legacy_api_client:Request ID is 6f8fd806-91e0-4a19-abcd-de91e24fde92
2025-07-11 11:05:05,981 INFO status has been updated to accepted
INFO:datapi.legacy_api_client:status has been updated to accepted
2025-07-11 11:05:20,416 INFO status has been updated to successful
INFO:datapi.legacy_api_client:status has been updated to successful
                                                                                       

  ✓ Download completed: C:\Users\ATHARV~1\AppData\Local\Temp\test2.zip
Extracting zip file: C:\Users\ATHARV~1\AppData\Local\Temp\test2.zip
Extracted NetCDF files:
  - C:\Users\ATHARV~1\AppData\Local\Temp\test2\data_stream-moda_stepType-avgad.nc
  - C:\Users\ATHARV~1\AppData\Local\Temp\test2\data_stream-moda_stepType-avgua.nc

Processing downloaded data:
- Found 2 file(s)
  Processing file 1/2: data_stream-moda_stepType-avgad.nc
  ✓ Loaded: Dimensions: Frozen({'valid_time': 12, 'latitude': 25, 'longitude': 73})
  Processing file 2/2: data_stream-moda_stepType-avgua.nc
  ✓ Loaded: Dimensions: Frozen({'valid_time': 12, 'latitude': 25, 'longitude': 73})
Starting optimized filtering process...
→ Extracting unique lat/lon coordinates from dataset...
✓ Found 1825 unique lat/lon combinations
→ Filtering unique coordinates against polygon...


✓ Coordinate filtering completed in 0.22 seconds
  - Points inside: 962
  - Points outside: 863
  - Percentage inside: 52.71%
→ Filtering original dataset using inside coordinates...
  Converting dataset to DataFrame...
  ✓ Converted to DataFrame with 43800 rows
  ✓ Created lookup set with 962 coordinate pairs
  Filtering DataFrame rows...
  ✓ Filtered from 43800 to 23088 rows
✓ Dataset filtering completed in 0.03 seconds

--- Final Filtering Results ---
Total processing time: 0.26 seconds
Final DataFrame shape: (23088, 7)
Rows in final dataset: 23088
AGGREGATING DATA (MONTHLY)
Aggregating data to monthly frequency...
Sum columns: ['tp']
Max columns: []
Min columns: []
Rate columns: []
Average columns: ['t2m']
Aggregation completed in:   0.04 seconds

Saving files to output directory: test2_output
  Saved final data to: test2_output\test2_monthly_data.csv
  Saved unique coordinates to: test2_output\test2_unique_latlongs.csv
  Saved raw data to: test2_output\test2_raw_data.csv

PROCESSI

In [ ]:
# eranest.era5ify_bbox(request_id, variables, start_date, end_date, north, south, east, west, dataset_type, pressure_levels, frequency, resolution)
# request_id : str, unique identifier for the request
# variables : list, list of variables to download
# start_date : string, start date of the data ('YYYY-M-D' or 'YYYY-MM-DD')
# end_date : string, end date of the data ('YYYY-M-D' or 'YYYY-MM-DD')
# north : float, northern boundary of the bounding box
# south : float, southern boundary of the bounding box
# east : float, eastern boundary of the bounding box
# west : float, western boundary of the bounding box
# dataset_type : str, type of dataset (single or pressure, for single level or pressure level datasets), optional (single by default)
# pressure_levels : list, list of strings of pressure levels to download (e.g., ["1000", "925", "850"]), optional (empty by default)
# frequency : str, frequency of the data (hourly, daily, weekly, monthly, yearly), optional (hourly by default)
# resolution : float, resolution of the data in degrees (0.1, 0.25, etc.), optional (0.25 by default)

In [4]:
df = eranest.era5ify_bbox(
    request_id="test3",
    variables=[
        "2m_temperature", 
        "total_precipitation"
    ],
    start_date="2024-01-1",
    end_date="2024-01-15",
    north = 30,
    south = 20,
    east = 80,
    west = 70,
    frequency="daily",
    resolution=0.25
)

✓ CDS API configuration is already set up and valid.

STARTING ERA5 SINGLE LEVEL PROCESSING
Request ID: test3
Variables: ['2m_temperature', 'total_precipitation']
Date Range: 2024-01-01 to 2024-01-15
Frequency: daily
Resolution: 0.25°
✓ All inputs validated successfully

--- Bounding Box ---
✓ Bounding Box calculated:
  North: 30.0000°
  South: 20.0000°
  East:  80.0000°
  West:  70.0000°
  Area:  10.0000° × 10.0000°

--- Processing Strategy ---
Using monthly dataset: False
Total days to process: 15
Max days per chunk: 14
Needs chunking: True
PROCESSING CHUNK 1/2
Date Range: 2024-01-01 to 2024-01-14
Variables:  2m_temperature, total_precipitation
  → Downloading ERA5 data (attempt 1/6)...


2025-07-11 11:08:03,271 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
INFO:datapi.legacy_api_client:[2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-07-11 11:08:03,951 INFO Request ID is 632dc9b2-c5ec-4b56-bf5c-67bbcc188c5d
INFO:datapi.legacy_api_client:Request ID is 632dc9b2-c5ec-4b56-bf5c-67bbcc188c5d
2025-07-11 11:08:04,155 INFO status has been updated to accepted
INFO:datapi.legacy_api_client:status has been updated to accepted
2025-07-11 11:08:13,167 INFO status has been updated to successful
INFO:datapi.legacy_api_client:status has been updated to successful
                                                                                         

  ✓ Download completed: C:\Users\ATHARV~1\AppData\Local\Temp\test3_chunk1.zip
Extracting zip file: C:\Users\ATHARV~1\AppData\Local\Temp\test3_chunk1.zip
Extracted NetCDF files:
  - C:\Users\ATHARV~1\AppData\Local\Temp\test3_chunk1\data_stream-oper_stepType-accum.nc
  - C:\Users\ATHARV~1\AppData\Local\Temp\test3_chunk1\data_stream-oper_stepType-instant.nc

Processing downloaded data:
- Found 2 file(s)
  Processing file 1/2: data_stream-oper_stepType-accum.nc
  ✓ Loaded: Dimensions: Frozen({'valid_time': 336, 'latitude': 41, 'longitude': 41})


  Processing file 2/2: data_stream-oper_stepType-instant.nc
  ✓ Loaded: Dimensions: Frozen({'valid_time': 336, 'latitude': 41, 'longitude': 41})
✓ Chunk completed in 17.2 seconds
PROCESSING CHUNK 2/2
Date Range: 2024-01-15 to 2024-01-15
Variables:  2m_temperature, total_precipitation
  → Downloading ERA5 data (attempt 1/6)...


2025-07-11 11:08:30,984 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
INFO:datapi.legacy_api_client:[2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-07-11 11:08:31,701 INFO Request ID is 7b42878f-3960-4d4d-82b4-a86093e3ff1f
INFO:datapi.legacy_api_client:Request ID is 7b42878f-3960-4d4d-82b4-a86093e3ff1f
2025-07-11 11:08:31,907 INFO status has been updated to accepted
INFO:datapi.legacy_api_client:status has been updated to accepted
2025-07-11 11:08:41,035 INFO status has been updated to running
INFO:datapi.legacy_api_client:status has been updated to running
2025-07-11 11:08:46,345 INFO status has been updated to successful
INFO:datapi.legacy_api_client:status has been updated to successful
                                                                                        

  ✓ Download completed: C:\Users\ATHARV~1\AppData\Local\Temp\test3_chunk2.zip
Extracting zip file: C:\Users\ATHARV~1\AppData\Local\Temp\test3_chunk2.zip
Extracted NetCDF files:
  - C:\Users\ATHARV~1\AppData\Local\Temp\test3_chunk2\data_stream-oper_stepType-accum.nc
  - C:\Users\ATHARV~1\AppData\Local\Temp\test3_chunk2\data_stream-oper_stepType-instant.nc

Processing downloaded data:
- Found 2 file(s)
  Processing file 1/2: data_stream-oper_stepType-accum.nc
  ✓ Loaded: Dimensions: Frozen({'valid_time': 24, 'latitude': 41, 'longitude': 41})
  Processing file 2/2: data_stream-oper_stepType-instant.nc
  ✓ Loaded: Dimensions: Frozen({'valid_time': 24, 'latitude': 41, 'longitude': 41})
✓ Chunk completed in 20.8 seconds
AGGREGATING DATA (DAILY)
Aggregating data to daily frequency...
Sum columns: ['tp']


Max columns: []
Min columns: []
Rate columns: []
Average columns: ['t2m']
Aggregation completed in:   0.13 seconds

Saving files to output directory: test3_output
  Saved final data to: test3_output\test3_daily_data.csv
  Saved unique coordinates to: test3_output\test3_unique_latlongs.csv
  Saved raw data to: test3_output\test3_raw_data.csv

PROCESSING COMPLETE

RESULTS SUMMARY:
----------------------------------------
Variables processed: 2
Time period:         2024-01-01 00:00:00 to 2024-01-15 00:00:00
Final output shape:  (15, 6)
Total complete processing time: 50.38 seconds

First 5 rows of aggregated data:
         tp         t2m  year  month  day        date
0  0.000059  289.480377  2024      1    1  2024-01-01
1  0.000158  288.823975  2024      1    2  2024-01-02
2  0.000437  288.676178  2024      1    3  2024-01-03
3  0.000333  288.236481  2024      1    4  2024-01-04
4  0.000649  288.469452  2024      1    5  2024-01-05

ERA5 SINGLE LEVEL PROCESSING COMPLETED SUCCESSFULLY


In [5]:
df = eranest.era5ify_bbox(
    request_id="test4",
    variables=[
        "temperature", 
        "relative_humidity"
    ],
    start_date="2024-01-1",
    end_date="2024-01-15",
    north = 30,
    south = 20,
    east = 80,
    west = 70,
    dataset_type="pressure",
    pressure_levels=["1000"],
    frequency="daily",
    resolution=0.25
)

✓ CDS API configuration is already set up and valid.

STARTING ERA5 PRESSURE LEVEL PROCESSING
Request ID: test4
Variables: ['temperature', 'relative_humidity']
Pressure Levels: ['1000']
Date Range: 2024-01-01 to 2024-01-15
Frequency: daily
Resolution: 0.25°
✓ All inputs validated successfully

--- Bounding Box ---
✓ Bounding Box calculated:
  North: 30.0000°
  South: 20.0000°
  East:  80.0000°
  West:  70.0000°
  Area:  10.0000° × 10.0000°

--- Processing Strategy ---
Using monthly dataset: False
Total days to process: 15
Max days per chunk: 14
Needs chunking: True
PROCESSING CHUNK 1/2
Date Range: 2024-01-01 to 2024-01-14
Variables:  temperature, relative_humidity
Levels:     1000
  → Downloading ERA5 data (attempt 1/6)...


2025-07-11 11:40:35,603 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
INFO:datapi.legacy_api_client:[2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-07-11 11:40:36,129 INFO Request ID is 39855550-5f97-4601-866e-4981fe8b9983
INFO:datapi.legacy_api_client:Request ID is 39855550-5f97-4601-866e-4981fe8b9983
2025-07-11 11:40:36,293 INFO status has been updated to accepted
INFO:datapi.legacy_api_client:status has been updated to accepted
2025-07-11 11:40:45,500 INFO status has been updated to running
INFO:datapi.legacy_api_client:status has been updated to running
2025-07-11 11:40:50,801 INFO status has been updated to successful
INFO:datapi.legacy_api_client:status has been updated to successful
                                                                                         

  ✓ Download completed: C:\Users\ATHARV~1\AppData\Local\Temp\test4_chunk1.nc
Copying NetCDF file: C:\Users\ATHARV~1\AppData\Local\Temp\test4_chunk1.nc
Extracted NetCDF files:
  - C:\Users\ATHARV~1\AppData\Local\Temp\test4_chunk1\test4_chunk1.nc

Processing downloaded data:
- Found 1 file(s)
  Processing file 1/1: test4_chunk1.nc
  ✓ Loaded: Dimensions: Frozen({'valid_time': 336, 'pressure_level': 1, 'latitude': 41, 'longitude': 41})


✓ Chunk completed in 19.5 seconds
PROCESSING CHUNK 2/2
Date Range: 2024-01-15 to 2024-01-15
Variables:  temperature, relative_humidity
Levels:     1000
  → Downloading ERA5 data (attempt 1/6)...


2025-07-11 11:41:06,147 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
INFO:datapi.legacy_api_client:[2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-07-11 11:41:06,568 INFO Request ID is 88fcc810-43c7-4c1e-b28e-887cd54cd123
INFO:datapi.legacy_api_client:Request ID is 88fcc810-43c7-4c1e-b28e-887cd54cd123
2025-07-11 11:41:06,707 INFO status has been updated to accepted
INFO:datapi.legacy_api_client:status has been updated to accepted
2025-07-11 11:41:15,541 INFO status has been updated to running
INFO:datapi.legacy_api_client:status has been updated to running
2025-07-11 11:41:21,061 INFO status has been updated to successful
INFO:datapi.legacy_api_client:status has been updated to successful
                                                                                      

  ✓ Download completed: C:\Users\ATHARV~1\AppData\Local\Temp\test4_chunk2.nc
Copying NetCDF file: C:\Users\ATHARV~1\AppData\Local\Temp\test4_chunk2.nc
Extracted NetCDF files:
  - C:\Users\ATHARV~1\AppData\Local\Temp\test4_chunk2\test4_chunk2.nc

Processing downloaded data:
- Found 1 file(s)
  Processing file 1/1: test4_chunk2.nc
  ✓ Loaded: Dimensions: Frozen({'valid_time': 24, 'pressure_level': 1, 'latitude': 41, 'longitude': 41})
✓ Chunk completed in 19.1 seconds
AGGREGATING DATA (DAILY)
Aggregating pressure level data to daily frequency...
Variables to average: ['t', 'r']
Including pressure_level in aggregation groups
Aggregation completed in:   0.06 seconds

Saving files to output directory: test4_output
  Saved final data to: test4_output\test4_daily_data.csv
  Saved unique coordinates to: test4_output\test4_unique_latlongs.csv


  Saved raw data to: test4_output\test4_raw_data.csv

PROCESSING COMPLETE

RESULTS SUMMARY:
----------------------------------------
Variables processed: 2
Time period:         2024-01-01 00:00:00 to 2024-01-15 00:00:00
Final output shape:  (15, 6)
Total complete processing time: 50.80 seconds

First 5 rows of aggregated data:
            t          r  pressure_level  year  month  day
0  292.125336  72.452385          1000.0  2024      1    1
1  291.697723  72.968575          1000.0  2024      1    2
2  291.161835  74.767265          1000.0  2024      1    3
3  291.061493  74.192192          1000.0  2024      1    4
4  290.956696  73.344673          1000.0  2024      1    5

ERA5 PRESSURE LEVEL PROCESSING COMPLETED SUCCESSFULLY


In [ ]:
# eranest.era5ify_point(request_id, variables, start_date, end_date, latitude, longitude, dataset_type, pressure_levels, frequency)
# request_id : str, unique identifier for the request
# variables : list, list of variables to download
# start_date : string, start date of the data ('YYYY-M-D' or 'YYYY-MM-DD')
# end_date : string, end date of the data ('YYYY-M-D' or 'YYYY-MM-DD')
# latitude : float, latitude of the point of interest
# longitude : float, longitude of the point of interest
# dataset_type : str, type of dataset (single or pressure, for single level or pressure level datasets), optional (single by default)
# pressure_levels : list, list of strings of pressure levels to download (e.g., ["1000", "925", "850"]), optional (empty by default)
# frequency : str, frequency of the data (hourly, daily, weekly, monthly, yearly), optional (hourly by default)

In [6]:
df = eranest.era5ify_point(
    request_id="test5",
    variables=[
        "2m_temperature", 
        "total_precipitation"
    ],
    start_date="2024-08-1",
    end_date="2024-08-14",
    latitude = 19.1331,
    longitude = 72.9151,
    frequency="daily",
    )

✓ CDS API configuration is already set up and valid.

STARTING ERA5 SINGLE LEVEL PROCESSING
Request ID: test5
Variables: ['2m_temperature', 'total_precipitation']
Date Range: 2024-08-01 to 2024-08-14
Frequency: daily
Resolution: 0.1°
GeoJSON File: C:\Users\ATHARV~1\AppData\Local\Temp\test5_temp_geojson.json
✓ All inputs validated successfully

--- Bounding Box ---
✓ Bounding Box calculated:
  North: 19.1931°
  South: 19.0731°
  East:  72.9786°
  West:  72.8516°
  Area:  0.1270° × 0.1200°




--- GeoJSON Mini Map ---

MINI MAP (72.85°W to 72.98°E, 19.07°S to 19.19°N):
┌─────────────────────────────────────────┐
│·········································│
│············■■■■■■■■■■■■■■■■■············│
│········■■■■■■■■■■■■■■■■■■■■■■■■■········│
│······■■■■■■■■■■■■■■■■■■■■■■■■■■■■■······│
│····■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■····│
│···■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■···│
│··■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■··│
│·■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■·│
│·■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■·│
│·■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■·│
│·■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■·│
│·■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■·│
│··■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■··│
│···■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■···│
│····■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■····│
│······■■■■■■■■■■■■■■■■■■■■■■■■■■■■■······│
│········■■■■■■■■■■■■■■■■■■■■■■■■■········│
│············■■■■■■■■■■■■■■■■■············│
│·········································│
└────────────────────────────────────────

2025-07-11 11:42:29,183 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
INFO:datapi.legacy_api_client:[2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.
2025-07-11 11:42:30,778 INFO Request ID is b8123ce9-f4cf-40a5-bb89-75be16c148cb
INFO:datapi.legacy_api_client:Request ID is b8123ce9-f4cf-40a5-bb89-75be16c148cb
2025-07-11 11:42:30,940 INFO status has been updated to accepted
INFO:datapi.legacy_api_client:status has been updated to accepted
2025-07-11 11:42:39,788 INFO status has been updated to successful
INFO:datapi.legacy_api_client:status has been updated to successful
                                                                                       

  ✓ Download completed: C:\Users\ATHARV~1\AppData\Local\Temp\test5.zip
Extracting zip file: C:\Users\ATHARV~1\AppData\Local\Temp\test5.zip
Extracted NetCDF files:
  - C:\Users\ATHARV~1\AppData\Local\Temp\test5\data_stream-oper_stepType-accum.nc
  - C:\Users\ATHARV~1\AppData\Local\Temp\test5\data_stream-oper_stepType-instant.nc

Processing downloaded data:
- Found 2 file(s)
  Processing file 1/2: data_stream-oper_stepType-accum.nc
  ✓ Loaded: Dimensions: Frozen({'valid_time': 336, 'latitude': 2, 'longitude': 2})
  Processing file 2/2: data_stream-oper_stepType-instant.nc
  ✓ Loaded: Dimensions: Frozen({'valid_time': 336, 'latitude': 2, 'longitude': 2})
Starting optimized filtering process...
→ Extracting unique lat/lon coordinates from dataset...
✓ Found 4 unique lat/lon combinations
→ Filtering unique coordinates against polygon...
✓ Coordinate filtering completed in 0.00 seconds
  - Points inside: 1
  - Points outside: 3
  - Percentage inside: 25.00%
→ Filtering original dataset using

In [ ]:
# eranest.search_variable(pattern, dataset_type)
# pattern : str, pattern to search for in variable names from dataset in dataset_type
# dataset_type :str,  Dataset type to search ("single", "pressure", "all", or any other registered dataset, all by dedault)

In [ ]:
eranest.search_variable("teMp ", dataset_type="all")


=== SEARCH RESULTS (ALL LEVELS) ===
Pattern: 'temp'
Variables found: 21

--- Temperature And Pressure ---
1. 2m_dewpoint_temperature (from single levels)
   Description: Dew point temperature at 2 meters above the surface, representing the temperature at which air becomes saturated with moisture. Unit: Kelvin (K).

2. 2m_temperature (from single levels)
   Description: Air temperature at 2 meters above the surface, typically used to represent surface-level weather conditions. Unit: Kelvin (K).

3. ice_temperature_layer_1 (from single levels)
   Description: Temperature of the top layer of sea ice. This layer is most affected by atmospheric conditions. Unit: Kelvin (K).

4. ice_temperature_layer_2 (from single levels)
   Description: Temperature of the second layer of sea ice, representing deeper internal ice temperatures. Unit: Kelvin (K).

5. ice_temperature_layer_3 (from single levels)
   Description: Temperature of the third layer of sea ice, indicating mid-level internal ice tempe

In [ ]:
# eranest.describe_variables(variable_names, dataset_type)
# variable_names : list, list of variable names to describe
# dataset_type : str, type of dataset (single or pressure, for single level or pressure level datasets)

In [ ]:
eranest.describe_variables(
    variable_names=["2m_temperature", "total_precipitation", "surface_pressure"], dataset_type="single"
)


=== Variable Descriptions (SINGLE LEVELS) ===

2m_temperature:
  Category: temperature_and_pressure
  Description: Air temperature at 2 meters above the surface, typically used to represent surface-level weather conditions. Unit: Kelvin (K).

total_precipitation:
  Category: precipitation_variables
  Description: Cumulative precipitation (convective + large-scale). Unit: meters (m) of water equivalent.

surface_pressure:
  Category: temperature_and_pressure
  Description: Atmospheric pressure at the surface of the Earth. Influenced by elevation and weather systems. Unit: Pascal (Pa).
